# User Workspace

This section completes the tutorial workflow by:
* Registering the user's processing results in their workspace
* Inspecting their product metadata via their Resource Catalogue
* Visualising their results via the Data Access services

## Access the workspace

In [ ]:
import requests
import datetime

In [ ]:
domain = "demo.eoepca.org"
#domain = "develop.eoepca.org"
workspace_prefix = "demo-user"

In [ ]:
USERNAME="user99"

In [ ]:
workspaces_endpoint = f"https://workspace-api.{domain}/workspaces"

Next step is to read the bucket credentials from the workspace data:

In [ ]:
actual_workspace_name = f'{workspace_prefix}-{USERNAME}'
response = requests.get(f"{workspaces_endpoint}/{actual_workspace_name}")
response.raise_for_status()

workspace_data = response.json()
bucket_name = workspace_data["storage"]["credentials"]["bucketname"]
s3_access = workspace_data["storage"]["credentials"]["access"]
s3_secret = workspace_data["storage"]["credentials"]["secret"]
workspace_data["storage"]["credentials"]['secret'] = "hidden_for_demo"
workspace_data["container_registry"]["password"] = "hidden_for_demo"
workspace_data

## Upload an application to the workspace

We will now upload an application package to the workspace:

In [ ]:
import boto3
#S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
S3_ENDPOINT = "https://minio.demo.eoepca.org"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)

In [ ]:
object = s3resource.Object(bucket_name, 'application-package/s-expression/s-expression-0_0_2.cwl')
result = object.put(Body=open('../data/s-expression-cwl.cwl', 'rb'))
res = result.get('ResponseMetadata')
if res.get('HTTPStatusCode') == 200:
    print('Application package uploaded successfully')
else:
    print('Application package not uploaded')

Now we register the application package:

In [ ]:
resource_url= f'{bucket_name}/application-package/s-expression/s-expression-0_0_2.cwl'
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/register",
    json={
        "type": "application",
        "url": resource_url,
    }
)
response.raise_for_status()
response

Now we check the registration result:

In [ ]:
workspace_catalogue_endpoint = f'https://resource-catalogue.{actual_workspace_name}.{domain}'

In [ ]:
import time
time.sleep(1)
response = requests.get(f"{workspace_catalogue_endpoint}/collections/metadata:main/items?type=application")
response.raise_for_status()
response.json()

## Upload data to the workspace

We will now upload a dataset to the workspace:

In [ ]:
import os
from os.path import relpath, join
import boto3

s3_client = boto3.client(
    service_name='s3',
    endpoint_url=S3_ENDPOINT,
    aws_access_key_id=s3_access,
    aws_secret_access_key=s3_secret,
)

data_root = "../data"
s3_prefix = "testdata"

for dirpath, _, filenames in os.walk(data_root):
    # print(dirpath, filenames)
    for filename in filenames:
        object_key = join(s3_prefix, relpath(join(dirpath, filename), data_root))
        s3_client.upload_file(join(dirpath, filename), bucket_name, object_key)

In [ ]:
import pystac

normalized_root = f"s3://{bucket_name}/{s3_prefix}/catalog.json"

catalog = pystac.read_file(f"{data_root}/catalog.json")
catalog.set_self_href(normalized_root)
catalog.normalize_hrefs(normalized_root)
catalog.make_all_asset_hrefs_absolute()
catalog.save(dest_href="../out/", catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED)
# catalog.normalize_and_save(, catalog_type=pystac.CatalogType.SELF_CONTAINED)

In [ ]:
# See if the files are uploaded
s3 = boto3.resource(
    's3',
    endpoint_url=S3_ENDPOINT,
    aws_access_key_id=s3_access,
    aws_secret_access_key=s3_secret
)

bucket = s3.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(obj)

A user workspace is a place for user data to be stored and served. It consists of a user bucket, a catalog and a data access system.

We will now proceed to register the uploaded files into the workspace system.

In [ ]:
register_url = f"{workspaces_endpoint}/{actual_workspace_name}/register"

response = requests.post(
    register_url,
    json={
        "type": "stac-item",
        "url": f"s3://{bucket_name}/{s3_prefix}/catalog.json",
    }
)
response.json()

In [ ]:
# helper function to do WMS GetMap requests and directly display the result in the notebook
def get_map(wms, layers, bbox, styles=None, size=None, srs='EPSG:4326', transparent=True, format="image/png", **kwargs):
    # if no specific size is passed, calculate one fitting the aspect ratio of the bbox
    if size is None:
        ratio = (bbox[3] - bbox[1])/(bbox[2] - bbox[0])
        width = 600
        height = int(ratio * width)
        size = (width, height)

    # qol helpers
    if isinstance(layers, str):
        layers = [layers]
    if isinstance(styles, str):
        styles = [styles]

    result = wms.getmap(
        layers=layers,
        styles=styles,
        size=size,
        srs=srs,
        bbox=bbox,
        format=format,
        transparent=transparent,
        **kwargs
    )
    return Image(result.read())

In [ ]:
from owslib.wms import WebMapService
from IPython.display import Image

data_access_base = f"data-access.{workspace_id}.{domain}"
ows_endpoint = f"https://{data_access_base}/ows"
# connect to the OWS endpoint using WMS
wms = WebMapService(ows_endpoint, version='1.3.0')
layer = wms.contents["S2B_MSIL2A_20190911T092029_N0213_R093_T34SFG_20190911T135255"]

bbox = layer.boundingBoxWGS84
layer.name

get_map(wms, "S2B_MSIL2A_20190911T092029_N0213_R093_T34SFG_20190911T135255__outlines", bbox)

In [ ]:
from tifffile import imread
from io import BytesIO

# convenience function to request a coverage and read the TIFF
def get_coverage(ows_endpoint, params):
    response = requests.get(ows_endpoint, params=params)
    response.raise_for_status()
    return imread(BytesIO(response.content))

In [ ]:
import matplotlib.pyplot as plt

ndvi = get_coverage(ows_endpoint, params={"service": "WCS", "version": "2.0.0", "request": "GetCoverage", "coverageid": "S2B_MSIL2A_20190911T092029_N0213_R093_T34SFG_20190911T135255_ndvi", "format": "image/tiff", "scaleFactor": "0.10"})
plt.imshow(ndvi, aspect="auto")

## Discover data with the workspace catalogue

In [ ]:
from owslib.csw import CatalogueServiceWeb

In [ ]:
workspace_catalogue_endpoint = f'https://resource-catalogue.{actual_workspace_name}.{domain}'
workspace_catalogue_csw = f'{workspace_catalogue_endpoint}/csw'

In [ ]:
import time
time.sleep(3)
response = requests.get(f"{workspace_catalogue_endpoint}/collections/metadata:main/items?type=application", headers=dict(f="json", **headers))
response.raise_for_status()
response.json()

In [ ]:
headers = {
    'Authorization': 'Bearer ' + user_id_token
}

List all records identifier, type and title in workspace catalogue

In [ ]:
csw = CatalogueServiceWeb(workspace_catalogue_csw, timeout=30,headers=headers)
csw.getrecords2(maxrecords=10)
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

In [ ]:
csw.records['s-expression'].references